In [1]:
import sys, os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
import warnings
warnings.filterwarnings("ignore")  # silence warnings
import pandas as pd
from datetime import datetime,date
from causalnex.structure import StructureModel
from causalnex.plots import plot_structure, NODE_STYLE, EDGE_STYLE
from causalnex.network import BayesianNetwork
from causalnex.structure.notears import from_pandas
from sklearn.cluster import KMeans
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Normalizer, MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
from IPython.display import Image
import plotly.express as px
import pickle
import matplotlib
matplotlib.style.use('fivethirtyeight')
import datetime
%matplotlib inline
sys.path.append(sys.path.append)
sys.path.insert(0,'../scripts/')


`Finding the Shortest Route`
- You can now make use of the OSMnx package together with the NetworkX package to find the route between two points.
> NetworkX is a Python package for the creation, manipulation, and study of the structure, dynamics, and functions of complex networks.

In [1]:
df_clean=pd.read_csv('../data/df_clean_final.csv')

NameError: name 'pd' is not defined